In [23]:
# def data_process(prcAll):
#     """
#     Convert raw price data into closeprice columns 
#     """
#     closePrice = []
#     for stock in range(50):
#         closePrice.extend(prcAll[stock])

#     dates = []
#     for stock in range(50):
#         for day in range(prcAll.shape[1]):
#             dates.append(day)

#     stocks = []
#     for stock in range(50):
#         stocks.extend([stock]*prcAll.shape[1])
#     full_data = pd.DataFrame({'date': dates, 'stock': stocks, 'closePrice': closePrice})

#     return full_data

In [1]:
def range_so_far(data):
    amp = []
    for j in range(50):
        single_stock_data = data[j]

        base_range = np.max(single_stock_data[-AMP_WINDOW:]) -  np.min(single_stock_data[-AMP_WINDOW:])
        amp.append(base_range)
    return amp

In [2]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


SHORT_TERM = 4              # Short term average
LONG_TERM = 15              # Long term average
PRICE_RANGE = 7             # The period to calculate price difference
AMP_WINDOW = 75            # The period to get stock amplitude

CHANGE_HOLDING = 500            # Price to change holding
AMP_LO_THRESHOLD = 7.5       # The threshold when price movement is considered low
AMP_HI_THRESHOLD = 1
PRICE_CHANGE_THRESHOLD = 0.01
MSE_THRESHOLD_2 = 0.04
SLOPE_THRESHOLD_2 = 2   # The threshold when price movement is considered high
MSE_THRESHOLD_1 = 0.05       # The threshold to control price volatility
SLOPE_THRESHOLD_1 = 0.05       # The threshold to control LR slope

nInst = 50
currentPos = np.zeros(nInst)
yesterday_sign = np.zeros(nInst)

def getMyPosition(prcSoFar):
	
    global currentPos, yesterday_sign

    currentPrices = prcSoFar[:,-1] # price of last day

    amp = range_so_far(prcSoFar)
	
    # Get long term and short term average prices
    for stock in range(50):
        single_stock_data = prcSoFar[stock]

        # Use short term and long term average to determine sign
        long_mean = single_stock_data[-LONG_TERM:].mean()
        short_mean = single_stock_data[-SHORT_TERM:].mean()
        today_sign = np.sign(short_mean - long_mean)

        # Use a price window to make decision
        n_day_diff = single_stock_data[-PRICE_RANGE] - single_stock_data[-1]
        n_day_range = np.max(single_stock_data[-PRICE_RANGE:]) - np.min(single_stock_data[-PRICE_RANGE:])
        # two_day_diff = single_stock_data.loc[day-3, 'closePrice'] - single_stock_data.loc[day-1, 'closePrice']

        # Calculate the MSE of price movement during the range
        n_day_gap = np.diff(single_stock_data[-PRICE_RANGE:])
        LR = LinearRegression(n_jobs=-1).fit(np.array(range(PRICE_RANGE-1)).reshape(-1, 1), n_day_gap.reshape(-1,1))
        n_day_mse = mean_squared_error(n_day_gap, LR.predict(np.array(range(PRICE_RANGE-1)).reshape(-1, 1)))
        
        if currentPos[stock] * n_day_range * np.sign(n_day_diff) > np.abs(currentPos[stock] * currentPrices[stock]) * PRICE_CHANGE_THRESHOLD and n_day_mse > np.abs(n_day_diff*MSE_THRESHOLD_2) or ((LR.coef_ * currentPos[stock] < 0)[0][0] and (np.abs(LR.coef_) > SLOPE_THRESHOLD_2)[0][0]):
            currentPos[stock] = 0

        elif np.abs(n_day_diff) <= amp[stock]/AMP_LO_THRESHOLD or (n_day_mse > np.abs(n_day_diff*MSE_THRESHOLD_1) and (np.abs(LR.coef_) < SLOPE_THRESHOLD_1)[0][0]):
            pass
            
        elif np.abs(n_day_diff) >= amp[stock]/AMP_HI_THRESHOLD:
            value = today_sign * CHANGE_HOLDING
            currentPos[stock] -= value//currentPrices[stock]
    
        else:
            value = today_sign * CHANGE_HOLDING
            currentPos[stock] += value//currentPrices[stock]
        
        yesterday_sign[stock] = today_sign
	
    return currentPos

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
# from main import getMyPosition as getPosition

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    #df=pd.read_csv(fn, sep='\s+', names=cols, header=None, index_col=0)
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./data/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

currentPos = np.zeros(nInst)

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(400,650): 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] #prcHist[:,t-1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        clipPos = np.clip(newPosOrig, -posLimits, posLimits)
        newPos = np.array([np.trunc(x) for x in clipPos])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))

    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)



(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)

Loaded 50 instruments for 750 days
Day 400 value: -10.67 todayPL: $-10.67 $-traded: 10669 return: -0.00100
Day 401 value: -20.23 todayPL: $-9.56 $-traded: 18492 return: -0.00109
Day 402 value: -21.34 todayPL: $-1.11 $-traded: 27519 return: -0.00078
Day 403 value: -16.30 todayPL: $5.04 $-traded: 37507 return: -0.00043
Day 404 value: 50.01 todayPL: $66.31 $-traded: 46485 return: 0.00108
Day 405 value: 51.66 todayPL: $1.65 $-traded: 53969 return: 0.00096
Day 406 value: 65.98 todayPL: $14.32 $-traded: 60466 return: 0.00109
Day 407 value: 211.42 todayPL: $145.44 $-traded: 66536 return: 0.00318
Day 408 value: 138.80 todayPL: $-72.63 $-traded: 74575 return: 0.00186
Day 409 value: 109.60 todayPL: $-29.19 $-traded: 81617 return: 0.00134
Day 410 value: 113.34 todayPL: $3.74 $-traded: 91128 return: 0.00124
Day 411 value: 128.87 todayPL: $15.53 $-traded: 98758 return: 0.00130
Day 412 value: 240.92 todayPL: $112.05 $-traded: 107210 return: 0.00225
Day 413 value: 263.12 todayPL: $22.20 $-traded: 114